In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

stock_name = "RELIANCE.NS"
stock_ = yf.Ticker(stock_name)

# get all stock info
all_info = stock_.info

# List of keys to filter
info_keys = ['symbol', 'shortName', 'industry', 'sector', 'longBusinessSummary','financialCurrency','marketCap', 'previousClose',
             'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'volume', 'averageVolume10days',
             'dividendRate', 'dividendYield', 'exDividendDate', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield',
             'payoutRatio', 'fiveYearAvgDividendYield','beta', 'trailingPE', 'forwardPE',
             'enterpriseValue', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'twoHundredDayAverage',
             'profitMargins',  'bookValue', 'priceToBook', 'trailingEps', 'forwardEps','enterpriseToRevenue', 'enterpriseToEbitda',
             'firstTradeDateEpochUtc', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 'totalCashPerShare',
             'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity',
             'revenuePerShare', 'returnOnAssets', 'returnOnEquity', 'freeCashflow', 'operatingCashflow',
             'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins','trailingPegRatio', 'netIncomeToCommon'
             ]

In [2]:
# Filter the dictionary
filtered_all_info = {key: all_info[key] for key in info_keys if key in all_info}

In [3]:
# filtered_all_info
# Returns a dictionary with all data files

In [4]:
#converting to DataFrame
data_all_info = pd.DataFrame([filtered_all_info])

In [5]:
# function to get 1Y and 5Y Growth Strategy
def find_growth(attribute:str):
  data = stock_.income_stmt
  _1Y_growth = (data.iloc[:,0].loc[attribute] - data.iloc[:,1].loc[attribute])/data.iloc[:,1].loc[attribute]
  _5Y_growth = (data.iloc[:,0].loc[attribute] - data.iloc[:,3].loc[attribute])/data.iloc[:,3].loc[attribute]
  _5Y_growth = _5Y_growth/3
  return _1Y_growth, _5Y_growth

In [6]:

# Feature Engineering 
_1Y_EBITDA_Growth, _5Y_EBITDA_Growth = find_growth(attribute='EBITDA')
_1Y_TotalRevenue_Growth, _5Y_TotalRevenue_Growth = find_growth(attribute='Total Revenue')
_1Y_TotalIncome_Growth, _5Y_TotalIncome_Growth = find_growth(attribute='Net Income')
_1Y_PreTaxIncome_Growth, _5Y_PreTaxIncome_Growth = find_growth(attribute='Pretax Income')

In [7]:
# Adding Features
data_all_info['netIncomeMargin'] = data_all_info['netIncomeToCommon']/data_all_info['totalRevenue']
data_all_info['% Away From 52High'] = (data_all_info['fiftyTwoWeekHigh']/data_all_info['currentPrice'])-1
data_all_info['% Away From 52Low'] = (data_all_info['currentPrice']/data_all_info['fiftyTwoWeekLow'])-1

data_all_info['1Y Total Revenue Growth'] = _1Y_TotalRevenue_Growth
data_all_info['5Y Total Revenue Growth'] = _5Y_TotalRevenue_Growth

data_all_info['1Y EBITDA Growth'] = _1Y_EBITDA_Growth
data_all_info['5Y EBITDA Growth'] = _5Y_EBITDA_Growth


data_all_info['1Y PreTax Income Growth'] = _1Y_PreTaxIncome_Growth
data_all_info['5Y PreTax Income Growth'] = _5Y_PreTaxIncome_Growth

data_all_info['1Y Net Income Growth'] = _1Y_TotalIncome_Growth
data_all_info['5Y Net Income Growth'] = _5Y_TotalIncome_Growth

In [8]:
data_all_info

,symbol,shortName,industry,sector,longBusinessSummary,financialCurrency,marketCap,previousClose,currentPrice,targetHighPrice,...,% Away From 52High,% Away From 52Low,1Y Total Revenue Growth,5Y Total Revenue Growth,1Y EBITDA Growth,5Y EBITDA Growth,1Y PreTax Income Growth,5Y PreTax Income Growth,1Y Net Income Growth,5Y Net Income Growth
0,RELIANCE.NS,RELIANCE INDUSTRIES LTD,Oil & Gas Refining & Marketing,Energy,Reliance Industries Limited engages in hydroca...,INR,17090068086784,1278.2,1262.9,1950.0,...,0.273893,0.037503,0.026462,0.309929,0.156705,0.258125,0.113572,0.2961,0.043762,0.139045


In [20]:
stock_.income_stmt.T

,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,...,Operating Expense,Other Operating Expenses,Selling General And Administration,Selling And Marketing Expense,General And Administrative Expense,Rent And Landing Fees,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
2024-03-31,4065480000.0,0.2455,1756750000000.0,16560000000.0,16560000000.0,696210000000.0,508320000000.0,6745990000000.0,1773310000000.0,1264990000000.0,...,1147990000000.0,530450000000.0,371420000000.0,209890000000.0,161530000000.0,6010000000.0,2264650000000.0,6745990000000.0,9010640000000.0,9010640000000.0
2023-03-31,-3185730000.0,0.2185,1547650000000.0,-14580000000.0,-14580000000.0,662840000000.0,403030000000.0,6718980000000.0,1533070000000.0,1130040000000.0,...,1039220000000.0,477350000000.0,316730000000.0,197360000000.0,119370000000.0,7290000000.0,2059370000000.0,6718980000000.0,8778350000000.0,8778350000000.0
2022-03-31,8188344000.0,0.2004,1222880000000.0,40860000000.0,40860000000.0,590440000000.0,297820000000.0,5316780000000.0,1263740000000.0,965920000000.0,...,855220000000.0,420900000000.0,248250000000.0,159420000000.0,88830000000.0,5780000000.0,1642850000000.0,5316780000000.0,6959630000000.0,6959630000000.0
2021-03-31,3659070000.0,0.0345,893340000000.0,106060000000.0,106060000000.0,491280000000.0,265720000000.0,3390610000000.0,999400000000.0,733680000000.0,...,736980000000.0,396010000000.0,196700000000.0,126780000000.0,69920000000.0,5990000000.0,1278630000000.0,3390610000000.0,4669240000000.0,4669240000000.0
2020-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
